In [ ]:
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
from paths import path_samoc, file_ex_ocn_ctrl
from regions import WGKP_area, sinking_area
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_DZ

In [ ]:
DZT = xr_DZ('ocn')

In [ ]:
# %%time
# # stacking files into one xr Dataset object
# for run in ['rcp']: #['ctrl', 'rcp']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM('ocn', run, 'monthly')):
#         if i==0:
#             start_year = y
            
#         da = xr.open_dataset(s, decode_times=False).XMXL[0,:,:]/100.
#         if m==1:
#             print(y)
#             da_new = da.copy()
#         else:
#             da_new = xr.where(da>da_new, da, da_new)
            
#         if m==12:
#             if y==start_year:
#                 da_max = da_new.copy()
#             else:
#                 da_max = xr.concat([da_new, da_max], dim='time')
                
# #         if y==start_year+2: break
#     da_max.to_netcdf(f'{path_samoc}/MXL/XMXL_yrly_{run}.nc')
# # 69 min

In [ ]:
ctrl_XMXL = xr.open_dataarray(f'{path_samoc}/MXL/XMXL_yrly_ctrl.nc', decode_times=False)/10
rcp_XMXL  = xr.open_dataarray(f'{path_samoc}/MXL/XMXL_yrly_rcp.nc' , decode_times=False)/10

# WGKP region
[90◦S, 50◦S] × [35◦W, 80◦E]

In [ ]:
ctrl_XMXL_mean = ctrl_XMXL.sel(WGKP_area).where(ctrl_XMXL.sel(WGKP_area)>0).mean(dim='time')
rcp_XMXL_mean  = rcp_XMXL.sel( WGKP_area).where(rcp_XMXL.sel( WGKP_area)>0).mean(dim='time')

ctrl_XMXL_max = ctrl_XMXL.sel(WGKP_area).where(ctrl_XMXL.sel(WGKP_area)>0).max(dim='time')
rcp_XMXL_max  = rcp_XMXL.sel( WGKP_area).where(rcp_XMXL.sel( WGKP_area)>0).max(dim='time')

In [ ]:
f, ax = plt.subplots(3, 2, figsize=(12,12))

DZT.sum(dim='z_t').sel(WGKP_area).plot(cmap=cmocean.cm.deep, ax=ax[0,0])

(ctrl_XMXL.sel(sinking_area).mean(dim=('nlat', 'nlon'))*(-1e-3)).plot(ax=ax[0,1], c='C0', label='CTRL')
(rcp_XMXL.sel( sinking_area).mean(dim=('nlat', 'nlon'))*(-1e-3)).plot(ax=ax[0,1], c='C1', label='RCP')

# (ctrl_XMXL.sel(sinking_area).max(dim=('nlat', 'nlon'))*(-1e-3)).plot(ax=ax[0,1], c='C0', ls='--', lw=1)
# (rcp_XMXL.sel( sinking_area).max(dim=('nlat', 'nlon'))*(-1e-3)).plot(ax=ax[0,1], c='C1', ls='--', lw=1)

(ctrl_XMXL.sel(WGKP_area).max(dim=('nlat', 'nlon'))*(-1e-3)).plot(ax=ax[0,1], c='C0', ls=':', lw=1)
(rcp_XMXL.sel( WGKP_area).max(dim=('nlat', 'nlon'))*(-1e-3)).plot(ax=ax[0,1], c='C1', ls=':', lw=1)
ax[0,1].set_ylim((-6,0))
ax[0,1].legend()

ctrl_XMXL_mean.plot(ax=ax[1,0], vmax=1200)
rcp_XMXL_mean.plot( ax=ax[1,1], vmax=1200)

ctrl_XMXL_max.plot(ax=ax[2,0], vmax=5200)
rcp_XMXL_max.plot( ax=ax[2,1], vmax=5200)

plt.tight_layout()